In [1]:
from adapters import AutoAdapterModel
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, LlamaTokenizer, LlamaForCausalLM
from peft import PeftModel
import transformers
import torch
import pandas as pd


def generate_prompt(instruction: str, input_ctxt: str = None) -> str:
    if input_ctxt:
        return f"""Below is an instruction that describes a task. Follow the instruction carefully and write an appropriate comprehensive reponse
        ### Instruction:
        {instruction}
        ### Input:
        {input_ctxt}
      	### Response:"""
    else:
      return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
    	 ### Instruction:
      {instruction}
       ### Response:"""


model = LlamaForCausalLM.from_pretrained("/bigwork/nhwpshaa/alpaca-native")
adapter_path = "/bigwork/nhwpshaa/Fine-tuned AlpacaNative/Jobs/30 epochs/LoRA config/" #specify the path for the adapter
adapter_name = model.load_adapter(adapter_path) #load the adapter
model.active_adapters = adapter_name #activate the adapter
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') #choose gpu if available
model.to(device)
tokenizer = LlamaTokenizer.from_pretrained("/bigwork/nhwpshaa/alpaca-native/")
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
instruction ="Act as an expert debater and generate an essay consisting of around 200 words for the following topic. Generate the essay while assuming only the 'con' stance."
input_ctxt ="Pollution"
sequences = pipeline(
    generate_prompt(instruction, input_ctxt),
    min_length=500,
    max_length=900,
    do_sample=True,
    top_k=90,
    num_return_sequences=4,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
        ### Instruction:
        Act as an expert debater and generate an essay consisting of around 200 words for the following topic. Generate the essay while assuming only the 'con' stance.
        ### Input:
        Pollution
      	### Response:Pollution is one of the greatest problems facing our planet today. It is created by human activities that discharge harmful substances into the environment, such as burning fossil fuels and the discharge of industrial waste. This destructive practice has wide-ranging effects, from air and water contamination to global warming. 

The most visible impact of pollution is the damage it causes to the environment. Water is particularly vulnerable to pollution, since it is a necessary medium for all forms of life. Chemicals such as oil, lead, and pesticides have the potential to conta

In [8]:
instruction ="Act as an expert debater and generate 10 arguments for and against the following topic."
input_ctxt ="federal banking reserve"
sequences = pipeline(
    generate_prompt(instruction, input_ctxt),
    min_length=500,
    max_length=900,
    do_sample=True,
    top_k=90,
    num_return_sequences=4,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
        ### Instruction:
        Act as an expert debater and generate 10 arguments for and against the following topic.
        ### Input:
        federal banking reserve
      	### Response:Arguments For Federal Banking Reserve:
1. The Federal Reserve System provides an important source of financial stability by promoting maximum employment and stable prices. 
2. The Federal Reserve ensures that there is sufficient quantities of money available to facilitate economic activity. 
3. The Federal Reserve provides an efficient payment system that helps to ensure timely and efficient settlement of transactions. 
4. The Federal Reserve encourages economic growth and stability through sound monetary policies. 
5. The Federal Reserve provides important services to depository institutions and the public, such as providing liquidit